In [1]:
from EVCS_agent_model import *

ModuleNotFoundError: No module named 'EVCS_agent_model'

In [6]:
def run_model(charge_per_tick, discharge_factor, no_homes, no_chargers, no_work, no_stores, scenario, no_ticks):
    """ runs the model for defined inputs and returns collected data as a pandas df""" 
    
    if scenario == 1:
        n = no_chargers - no_homes 
    if scenario == 2: 
        n = no_chargers
        
    no_agents = no_homes  #note that number of homes = number of EV agents (one agent per household)

    MW = Model_World(no_homes = no_homes, no_work = no_work, no_stores = no_stores, no_ticks = no_ticks)
    Model_Graph = MW.G 
    trip_lists = MW.get_trip_lists()
    charger_placement = MW.get_charger_placement(scenario, int(n))
    model = EVCSModel(no_agents=no_agents, no_chargers=no_chargers, charger_placement =charger_placement, no_ticks = no_ticks, G=Model_Graph, trip_lists = trip_lists)
    
    for _ in range(no_ticks):
        model.step()
    
    data_collection = model.datacollector.get_model_vars_dataframe()

    return data_collection

In [7]:
def create_figures(data_collection): 
    """plots data collection values against the ticks for a single model run """
    for value in data_collection.columns:
        plt.figure(figsize=(8, 5))
        plt.plot(data_collection[value])
        plt.xlabel("Tick")
        plt.ylabel(str(value))
        plt.title(str(value) + " over time")
        plt.show()

#### Sensitivity Analysis: Run model repeatedly with varied inputs and plot affect

In [8]:
#Baseline model inputs
charge_per_tick = 20
discharge_factor = 30
flag = False

no_homes = 50
no_work = 5
no_stores = 10
no_chargers = 55

scenario = 1
no_ticks = 336

In [9]:
def plot_sensitivity_analysis(item, df): 
    for value in df.columns:
        plt.figure(figsize=(8, 5))
        
        scenario_1 = df[df["Scenario"] == 1]
        scenario_2 = df[df["Scenario"] == 2]
        
        plt.plot(scenario_1[item],scenario_1[value], label = "Scenario 1")
        plt.plot(scenario_2[item], scenario_2[value],label = "Scenario 2")
        plt.legend()
        plt.xlabel(item)
        plt.ylabel(str(value))
        plt.title("Impact of " + item + " on " + str(value))
        plt.show()      

In [10]:
#Vary number of Chargers
number_chargers = np.linspace(50, 75, 6)
no_homes = 50

columns = ['Average SOC', 'EV trips complete', 'Amount charged', 'Instances of insufficient charge', 'Length of queue']
data = {item:[] for item in columns}  
data["Number of Chargers"] = []
data["Scenario"] = []
                      
for no_chargers in number_chargers: 
    for scenario in [1,2]:
        data_collection = run_model(charge_per_tick, discharge_factor, no_homes, int(no_chargers), no_work, no_stores, scenario, no_ticks)
        data["Scenario"].append(scenario)   
        data["Number of Chargers"].append(no_chargers)
        data["Average SOC"].append(data_collection["Average SOC"].mean())
        data["EV trips complete"].append(data_collection["EV trips complete"].max())
        data["Amount charged"].append(data_collection["Amount charged"].max())
        data["Instances of insufficient charge"].append(data_collection["Instances of insufficient charge"].max())
        data["Length of queue"].append(data_collection["Length of queue"].mean())
    
    
df = pd.DataFrame(data)

NameError: name 'np' is not defined

In [11]:
plot_sensitivity_analysis("Number of Chargers", df)

NameError: name 'df' is not defined

In [ ]:
#Vary number of homes (agents)
no_chargers = 100

number_homes = np.linspace(10,100, 10) 
columns = ['Average SOC', 'EV trips complete', 'Amount charged', 'Instances of insufficient charge', 'Length of queue']
data = {item:[] for item in columns}  
data["Number of EV agents"] = []
data["Scenario"] = []
                      
for no_homes in number_homes: 
    for scenario in [1,2]:
        data_collection = run_model(charge_per_tick, discharge_factor, int(no_homes), no_chargers, no_work, no_stores, scenario, no_ticks)
        data["Scenario"].append(scenario)   
        data["Number of EV agents"].append(no_homes)
        data["Average SOC"].append(data_collection["Average SOC"].mean())
        data["EV trips complete"].append(data_collection["EV trips complete"].max())
        data["Amount charged"].append(data_collection["Amount charged"].max())
        data["Instances of insufficient charge"].append(data_collection["Instances of insufficient charge"].max())
        data["Length of queue"].append(data_collection["Length of queue"].mean())
    
    
df = pd.DataFrame(data)


In [ ]:
plot_sensitivity_analysis("Number of EV agents", df)

In [ ]:
#Vary charging rate
discharge_factor = 30

no_homes = 50
no_work = 5
no_stores = 10
no_chargers = 55

scenario = 1
no_ticks = 336

charge_rate = np.linspace(5,35, 7) 

columns = ['Average SOC', 'EV trips complete', 'Amount charged', 'Instances of insufficient charge', 'Length of queue']
data = {item:[] for item in columns}  
data["Charge Rate"] = []
data["Scenario"] = []
                      
for charge_per_tick in charge_rate: 
    for scenario in [1,2]:
        data_collection = run_model(int(charge_per_tick), discharge_factor, no_homes, no_chargers, no_work, no_stores, scenario, no_ticks)
        data["Scenario"].append(scenario)   
        data["Charge Rate"].append(charge_per_tick)
        data["Average SOC"].append(data_collection["Average SOC"].mean())
        data["EV trips complete"].append(data_collection["EV trips complete"].max())
        data["Amount charged"].append(data_collection["Amount charged"].max())
        data["Instances of insufficient charge"].append(data_collection["Instances of insufficient charge"].max())
        data["Length of queue"].append(data_collection["Length of queue"].mean())
    
    
df = pd.DataFrame(data)


In [ ]:
plot_sensitivity_analysis("Charge Rate", df)